# Tutorial: Benchmark de Difusión y Maximización de Influencia

Este cuaderno actualiza el funcionamiento de `difusion_lib` para realizar estudios comparativos. El objetivo ahora no es solo pelar la red, sino comparar la eficiencia del algoritmo de **Pelado (PEL)** frente a estándares de la industria como **CELF** (Cost-Effective Lazy Forwarding) y **RIS** (Reverse Influence Sampling).

## 1. Arquitectura del Sistema de Simulación

La librería ahora integra capacidades de análisis comparativo:

### A. Módulos de Maximización de Influencia
- **`AnalizadorCELF`**: Implementa la optimización submodular para encontrar semillas de influencia de forma codiciosa pero eficiente.
- **`AnalizadorRIS`**: Utiliza conjuntos alcanzables invertidos para estimar la influencia en grafos de gran escala.
- **`ControladorPelado`**: Nuestro método propietario que utiliza la descomposición por difusión de masa para identificar el núcleo (core) de la red.

### B. `simulacion.py` (Orquestador de Baterías)
El nuevo motor de pruebas permite:
1. **Multimetodología**: Ejecutar PEL, CELF y RIS sobre el mismo grafo exactamente.
2. **Configuraciones Dinámicas**: Probar múltiples topologías (Mallas, Cascadas, SBM) en una sola ejecución.
3. **Métricas de Rendimiento**: Compara no solo el alcance (nodos mojados), sino también el tiempo de ejecución de cada método.

In [ ]:
import os
import pandas as pd
import networkx as nx
import time
from datetime import datetime

# Importación de la librería principal y módulos de análisis
from difusion_lib import *

# Configuración de tipos de grafos para el estudio
configuraciones_estudio_peq = [
    {'tipo': 'malla_netlogo', 'params': {'dim': 25, 'link_chance': 60}},
    {'tipo': 'cascada', 'params': {'n_bloques': 15, 'nodos_por_bloque': 50}},
    {'tipo': 'flujo_libre', 'params': {'n_nodos': 600}},
    {'tipo': 'sbm', 'params': {'n_total': 654, 'n_grupos': 8}},
    {'tipo': 'red_social_realista', 'params': {'n_users': 500, 'm_neighbors': 2, 'p_triangle': 0.3, 'ratio_mutual': 0.5}}
]

## 2. Definición del Motor de Batería Masiva

La función `ejecutar_bateria_masiva` ahora actúa como un entorno de benchmark científico. Para cada grafo generado, extrae las semillas candidatas mediante los tres métodos y mide su capacidad de infectar la red bajo el mismo modelo de difusión.

In [ ]:
def cantidad_nodos_mojados(record):
    return len(set(record))

def ejecutar_bateria_masiva(
    metodos=['pel', 'celf', 'ris'],
    configuraciones_grafos=None, 
    n_simulaciones=1, 
    master_folder="simulaciones", 
    tasa_difusion=0.4, 
    iteraciones_difusion=350,
    generar_visualizaciones=True,
    **kwargs_extra
):
    # Mapeo interno de generadores
    mapeo_generadores = {
        'malla_netlogo': GeneradorRedes.generar_malla_estocastica_netlogo,
        'cascada': GeneradorRedes.generar_cascada_estricta,
        'flujo_libre': GeneradorRedes.generar_flujo_libre_escala,
        'sbm': GeneradorRedes.generar_sbm_estocastico,
        'red_social_realista' : GeneradorRedes.generar_red_social_realista
    }

    if not os.path.exists(master_folder): os.makedirs(master_folder)
    resumen_global = []

    for config in configuraciones_grafos:
        tipo = config['tipo']
        params = config.get('params', {})
        print(f"\n>>> Iniciando Experimento: {tipo.upper()}")
        
        resumen_metricas = []
        for i in range(n_simulaciones):
            # 1. Generación
            G = mapeo_generadores[tipo](**params)
            G_original = G[0] if isinstance(G, tuple) else G
            
            # 2. Extracción de Semillas y Medición de Tiempos
            # [Lógica de PEL, CELF y RIS omitida por brevedad en este bloque, ver simulacion.py]
            
            # Registro de métricas comparativas
            resumen_metricas.append({
                "Simulacion_ID": f"Sim_{i+1}",
                "Tipo_Grafo": tipo,
                "Ratio_Mojados_PEL": 0.0, # Ejemplo de placeholder
                "Ratio_Mojados_CELF": 0.0,
                "Ratio_Mojados_RIS": 0.0
            })
            
        df_tipo = pd.DataFrame(resumen_metricas)
        resumen_global.append(df_tipo)

    print("=== BATERÍA COMPLETADA ===")

## 3. Ejecución del Experimento Multigrafo

Lanzamos el estudio sobre las 5 topologías definidas. Esto permitirá ver, por ejemplo, si el método PEL es más rápido que RIS en grafos tipo cascada o si CELF mantiene un mejor ratio en redes sociales.

In [ ]:
marca_tiempo = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
path_estudio = f"simulaciones/Estudio_Multigrafo_{marca_tiempo}"

ejecutar_bateria_masiva(
    metodos=['pel', 'celf', 'ris'],
    configuraciones_grafos=configuraciones_estudio_peq,
    n_simulaciones=1, # Para prueba rápida
    master_folder=path_estudio,
    tasa_difusion=0.4,
    iteraciones_difusion=350,
    generar_visualizaciones=True
)

## 4. Análisis de Resultados Comparativos

Al finalizar, el sistema genera el archivo `Promedios_Globales_Bateria.csv`. Al analizarlo, presta atención a:

1. **Tiempo_Eje_PEL vs CELF**: En redes grandes, CELF suele volverse prohibitivo. Observa si PEL mantiene un tiempo lineal.
2. **Ratio_Mojados**: Mide la calidad de las semillas. Un ratio de 1.0 indica que las semillas seleccionadas lograron cubrir toda la red.
3. **Uso de CFC**: Si activaste `usar_cfc`, verifica si la descomposición en Componentes Fuertemente Conectados mejoró la velocidad del pelado en grafos dirigidos complejos.